<center>

# ~~~ Changing bases, non-real braket identities, and transformation to Slater integrals ~~~

</center>


<center><img src="./images/qdef-banner.png" style="width: 500px">



<div style="max-width:600px; word-wrap:break-word;"

- Coded function <span style="color: #ff0000"> braket_basis_change </span> that changes bases in four symbol brakets. This allows replacing the abstract bases for irreducible representations into symmetry adapted spherical harmonics.

- Modify <span style="color: #ff0000">braket_identities </span> to allow for relaxing the condition of real wavefunctions.

- Code the function <span style="color: #ff0000">to_slater_params</span> that interprets a sum of four symbol brakets, written in terms of d-orbitals, in terms of Slater integrals.

- Compare results to those found in STK.

</div>

</center>




In [1]:
%load_ext line_profiler
%load_ext autoreload
# provide cell timings
%load_ext autotime
%autoreload 2
import sympy as sp
import numpy as np
%config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from misc import *
from IPython.display import display, Math, Latex
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
import time
from itertools import product
from itertools import permutations
import re
from sympy import Eijk as εijk
from sympy.functions.special.tensor_functions import KroneckerDelta

Reloading /Users/juan/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


# Functions

In [2]:
def braket_basis_change(braket, basis_changer):
    '''
    Take  a  qet,  understood  as  a  four  symbol braket, and a
    dictionary  that  maps  the  current basis to a new one, and
    return  the resulting expression for the new braket in terms
    of  the new basis. All throughout it is assumed that between
    the given braket there is an implicit operator.

    Parameters
    ----------
    braket   (qdefcore.Qet)
    basis_changer (dict):  keys being  equal  to single electron
    quantum  symbols  and  values  to  qets  that  determine the
    superposition of the new basis to which this vector is being
    mapped to. The keys of the dictionary need not  include  all
    the quantum symbols included in the qet.

    Returns
    -------
    new_braket (qdefcore.Qet)

    Example
    -------

    braket = Qet({(1,2,3,4): 5,
                (8,4,3,1): 1})
    basis_change = {1: Qet({(8,): sp.I})}
    print(braket_basis_change(braket, basis_change))
    >> {(8, 2, 3, 4): -5*I, (8, 4, 3, 8): I}

    '''

    new_braket = Qet({})
    for k, v in braket.dict.items():
        βi, βj, βk, βl = [(Qet({(γ,):1}) if γ not in basis_changer else basis_changer[γ]) for γ in k]
        βi, βj = βi.dual(), βj.dual()
        γiγjγkγl = ((βi * βj) * βk) * βl
        new_braket = new_braket + (v*γiγjγkγl)
    return new_braket

def to_slater_params(qnums, coeff):
    '''
    This function will take a set of qnums that are assumed to be l1, m1, l2, m2, l1p, m1p, l2p, m2p
    and will return a set of qnums that correspond to slater integrals and corresponding coefficients.
    '''
    if len(qnums) == 8:
        l1, m1, l2, m2, l1p, m1p, l2p, m2p = qnums
    else:
        m1, m2, m1p, m2p = qnums
        l1, l2, l1p, l2p = 2, 2, 2, 2
    if KroneckerDelta(m1+m2,m1p+m2p):
        if (m1-m1p) % 2 == 0:
            phase = 1
        else:
            phase = -1
        new_dict = {}
        for k in range(6):
            key = sp.Symbol('F^{(%d)}' % k)
            c1 = threeHarmonicIntegral(l1,   m1,
                                        k,   (m1-m1p),
                                        l1p, m1p) * sp.sqrt((4*sp.pi) / (2*k+1))
            c2 = threeHarmonicIntegral(l2,  m2,
                                       k,   (m2-m2p),
                                       l2p, m2p) * sp.sqrt((4*sp.pi) / (2*k+1))
            val = phase*coeff*c1*c2
            if val:
                # if k == 4:
                #     print(k,'|',val,c1,c2,coeff, phase)
                #     print(qnums)
                #     print(coeff)
                new_dict[key] = val
        return new_dict
    else:
        # print("s")
        return {}

A, B, C = sp.symbols('A B C')
subs = {sp.Symbol('F_4') : sp.Symbol('F^(4)')/441,
        sp.Symbol('F_0'): sp.Symbol('F^(0)'),
        sp.Symbol('F_2'): sp.Symbol('F^(2)')/49}
A = sp.Symbol('F_0') - 49 * sp.Symbol('F_4')
A = A.subs(subs)
B = sp.Symbol('F_2') - 5 * sp.Symbol('F_4')
B = B.subs(subs)
C = 35*sp.Symbol('F_4')
C = C.subs(subs)

In [3]:
four_real_var_ids, four_symbol_ids, two_real_var_ids, two_symbol_ids = braket_identities('O', assume_real = False)

running updated function ...
O Creating all 4-symbol identities ...
O Refining set of identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 4-symbol brakets ...
O Creating a dictionary with all the 4-symbol replacements ...
O Creating all 2-symbol identities ...
O Creating set of 2 symbol identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 2-symbol brakets ...
O Creating a dictionary with all the 2-symbol replacements ...


In [7]:
group_label = 'O'
l = 2
group = CPGs.get_group_by_label(group_label)
irreps = group.irrep_labels
sym_bases = {k: v[l] for k,v in symmetry_bases[group_label][1].items() if v[l]}
#####################################################
#####################################################
#####################################################
# CAUTION: I'm assuming that the expression that I calculated
# here for a symmetry adapted basis component matches in order
# to the labels for the components of each irrep
#####################################################
#####################################################
#####################################################
component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
basis_change = {}
for irrep, qets in sym_bases.items():
    components = component_labels[irrep]
    if isinstance(qets, list):
        qets = qets[0]
    for component, qet in zip(components, qets):
        basis_change[component] = qet


In [8]:
def simplifier(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        if k in four_real_var_ids:
            simp_ket += Qet({four_real_var_ids[k]:v})
        else:
            simp_ket += Qet({k:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in four_symbol_ids:
            true_qet += v*Qet(four_symbol_ids[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

In [9]:
ground_truth = {
    (0, sp.Symbol('A_1')): (A + 4*B + 3*C) + 2*(3*B +C),
    (0, sp.Symbol('E')): (A + 4*B + 3*C) - 1*(3*B +C),
    (0, sp.Symbol('T_2')): (A - 2*B + C) + 1*(3*B +C),
    (1, sp.Symbol('T_1')): (A - 2*B + C) - 1*(3*B +C),
}

t23 = CrystalElectronsSCoupling('O',[sp.Symbol('T_2')]*2)

terms = {}
for wave_key, wave_qet in t23.equiv_waves.items():
    aterm = wave_key.terms[-1]
    if aterm not in terms:
        terms[aterm] = {}
    terms[aterm][wave_key] = wave_qet
terms = OrderedDict(sorted(list(terms.items()), key=lambda x: x[0][0]))

for term, term_qets in terms.items():
    display(sp.Symbol('+'*10))
    term_symbol = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
    a_key = list(term_qets.keys())[0]
    a_qet = term_qets[a_key]
    matrix_element = double_electron_braket(a_qet,a_qet)
    matrix_element = simplifier(matrix_element)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
    matrix_element = braket_basis_change(matrix_element, basis_change)
    # display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
    # display(matrix_element.as_braket())
    matrix_element = matrix_element.apply(to_slater_params)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_symbol_sum()))))
    display(ground_truth[term])


++++++++++

<IPython.core.display.Math object>

<IPython.core.display.Math object>

F^(0) + 10*F^(2)/49 + 76*F^(4)/441

++++++++++

...

F^(0) + F^(2)/49 + 16*F^(4)/441

++++++++++

<IPython.core.display.Math object>

<IPython.core.display.Math object>

F^(0) - 5*F^(2)/49 - 8*F^(4)/147

In [10]:
t23 = CrystalElectronsSCoupling('O',[sp.Symbol('E')]*4)

terms = {}
for wave_key, wave_qet in t23.equiv_waves.items():
    aterm = wave_key.terms[-1]
    if aterm not in terms:
        terms[aterm] = {}
    terms[aterm][wave_key] = wave_qet
terms = OrderedDict(sorted(list(terms.items()), key=lambda x: x[0][0]))

for term, term_qets in terms.items():
    display(sp.Symbol('+'*10))
    term_symbol = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
    a_key = list(term_qets.keys())[0]
    a_qet = term_qets[a_key]
    matrix_element = double_electron_braket(a_qet,a_qet)
    matrix_element = simplifier(matrix_element)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
    matrix_element = braket_basis_change(matrix_element, basis_change)
    # display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
    # display(matrix_element.as_braket())
    matrix_element = matrix_element.apply(to_slater_params)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_symbol_sum()))))
    # display(ground_truth[term])

++++++++++

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
t23 = CrystalElectronsLLcoupling('O',(sp.Symbol('E'),2), (sp.Symbol('T_2'),4))

terms = {}
for wave_key, wave_qet in t23.equiv_waves.items():
    aterm = wave_key.terms[-1]
    if aterm not in terms:
        terms[aterm] = {}
    terms[aterm][wave_key] = wave_qet
terms = OrderedDict(sorted(list(terms.items()), key=lambda x: x[0][0]))

for term, term_qets in terms.items():
    display(sp.Symbol('+'*10))
    term_symbol = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
    a_key = list(term_qets.keys())[0]
    a_qet = term_qets[a_key]
    matrix_element = double_electron_braket(a_qet,a_qet)
    matrix_element = simplifier(matrix_element)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
    matrix_element = braket_basis_change(matrix_element, basis_change)
    # display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
    # display(matrix_element.as_braket())
    matrix_element = matrix_element.apply(to_slater_params)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_symbol_sum()))))
    # display(ground_truth[term])

++++++++++

<IPython.core.display.Math object>

<IPython.core.display.Math object>

++++++++++

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

++++++++++

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
# Qet(to_slater_params((  2,2,
#                         2,-2,
#                         2,-2,
#                         2,2),1)).as_symbol_sum()
# l = 1
# lp = 2
# m = 1
# mp = -2
# k = 3
# mk = m-mp
# threeHarmonicIntegral(l,m,k,mk,lp,mp) * sp.sqrt(4*sp.pi) / sp.sqrt(2*k+1) * 7 * sp.sqrt(5)

In [ ]:
# qets = list(t23.equiv_waves.values())
# aqet = qets[0]
# abraket = double_electron_braket(aqet,aqet)
# current_basis = set()
# for qet in qets:
#     for k in qet.dict.keys():
#         current_basis.update(k)
# basis_change = {}
# new_basis_symbols = [sp.Symbol(r'\omega_{%d}' % d) for d in range(5)]
# for basis_vec in current_basis:
#     this_change = {(bv,): np.random.randint(0,10) for bv in new_basis_symbols}
#     this_qet = Qet(this_change)
#     normalizer = sp.S(1)/this_qet.norm()
#     this_qet = normalizer * this_qet
#     basis_change[basis_vec] = this_qet
# new_braket = braket_basis_change(abraket, basis_change)
# new_braket.as_ket()

# group_label = 'O'
# l = 2
# group = CPGs.get_group_by_label(group_label)
# irreps = group.irrep_labels
# sym_bases = {k: v[l] for k,v in symmetry_bases[group_label][1].items() if v[l]}
# #####################################################
# #####################################################
# #####################################################
# # CAUTION: I'm assuming that the expression that I calculated
# # here for a symmetry adapted basis component matches in order
# # to the labels for the components of each irrep
# #####################################################
# #####################################################
# #####################################################
# component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
# basis_change = {}
# for irrep, qets in sym_bases.items():
#     components = component_labels[irrep]
#     print(irrep)
#     print(len(qets))
#     if isinstance(qets, list):
#         qets = qets[0]
#     for component, qet in zip(components, qets):
#         basis_change[component] = qet
# # basis_change[sp.Symbol(r'{\xi}')] = sp.I * basis_change[sp.Symbol(r'{\xi}')]
# # basis_change

# real_var_simplifiers = {irc:[] for irc in [1,2,3]}

# real_var_full_simplifier = {}
# for ircombo in real_var_simplifiers:
#     real_var_full_simplifier.update(real_var_simplifiers[ircombo])

In [ ]:
# # This should equal 2.105
# huh = braket_basis_change(Qet({(sp.Symbol(r'{\xi}'), sp.Symbol(r'{\xi}'), sp.Symbol(r'{\xi}'), sp.Symbol(r'{\xi}')):1}),basis_change)
# new_huh = Qet()
# for k,v in huh.dict.items():
#     m1, m2, m1p, m2p = k[1],k[3],k[5],k[7]
#     if (m1+m2 == m1p+m2p):
#         new_huh = new_huh + Qet({(m1,m2,m1p,m2p):v})
# new_huh.as_braket()
# new_huh.apply(to_slater_params).as_symbol_sum()
# # and it does

In [ ]:
# # def to_slater_params(qnums, coeff):
# #     '''
# #     This function will take a set of qnums that are assumed to be l1, m1, l2, m2, l1p, m1p, l2p, m2p
# #     and will return a set of qnums that correspond to slater integrals and corresponding coefficients.
# #     '''
# #     if len(qnums) == 8:
# #         l1, m1, l2, m2, l1p, m1p, l2p, m2p = qnums
# #     else:
# #         m1, m2, m1p, m2p = qnums
# #         l1, l2, l1p, l2p = 2, 2, 2, 2
# #     if KroneckerDelta(m1+m2,m1p+m2p):
# #         if (m1-m1p) % 2 == 0:
# #             phase = 1
# #         else:
# #             phase = -1
# #         new_dict = {}
# #         for k in range(6):
# #             key = sp.Symbol('F^{(%d)}' % k)
# #             c1 = threeHarmonicIntegral(l1,   m1,
# #                                         k,   (m1-m1p),
# #                                         l1p, m1p) * sp.sqrt((4*sp.pi) / (2*k+1))
# #             c2 = threeHarmonicIntegral(l2,  m2,
# #                                        k,   (m2-m2p),
# #                                        l2p, m2p) * sp.sqrt((4*sp.pi) / (2*k+1))
# #             val = phase*coeff*c1*c2
# #             if val:
# #                 if k == 4:
# #                     print(k,'|',val,c1,c2,coeff, phase)
# #                     print(qnums)
# #                     print(coeff)
# #                 new_dict[key] = val
# #         return new_dict
# #     else:
# #         print("s")
# #         return {}
# # this should equal F0 - 5/49 F^2 -74/441 F^4
# matrix_element = double_electron_braket(a_qet,a_qet)
# matrix_element = simplifier(matrix_element)
# display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
# matrix_element = braket_basis_change(matrix_element, basis_change)
# # display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
# # display(matrix_element.as_braket())
# matrix_element = matrix_element.apply(to_slater_params)
# display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_symbol_sum()))))

In [ ]:
# # when i go through all of table 2.5
# # all the results that I compute match
# # with the table
# # def to_slater_params(qnums, coeff):
# #     '''
# #     This function will take a set of qnums that are assumed to be l1, m1, l2, m2, l1p, m1p, l2p, m2p
# #     and will return a set of qnums that correspond to slater integrals and corresponding coefficients.
# #     '''
# #     if len(qnums) == 8:
# #         l1, m1, l2, m2, l1p, m1p, l2p, m2p = qnums
# #     else:
# #         m1, m2, m1p, m2p = qnums
# #         l1, l2, l1p, l2p = 2, 2, 2, 2
# #     if KroneckerDelta(m1+m2,m1p+m2p):
# #         if (m1-m1p) % 2 == 0:
# #             phase = 1
# #         else:
# #             phase = -1
# #         new_dict = {}
# #         for k in range(6):
# #             key = sp.Symbol('F^{(%d)}' % k)
# #             c1 = threeHarmonicIntegral(l1,   m1,
# #                                         k,   (m1-m1p),
# #                                         l1p, m1p) * sp.sqrt((4*sp.pi) / (2*k+1))
# #             c2 = threeHarmonicIntegral(l2,  m2,
# #                                        k,   (m2-m2p),
# #                                        l2p, m2p) * sp.sqrt((4*sp.pi) / (2*k+1))
# #             val = phase*coeff*c1*c2
# #             if val:
# #                 # if k == 4:
# #                 #     print(k,'|',val,c1,c2,coeff, phase)
# #                 #     print(qnums)
# #                 #     print(coeff)
# #                 new_dict[key] = val
# #         return new_dict
# #     else:
# #         # print("s")
# #         return {}

# A, B, C = sp.symbols('A B C')
# subs = {sp.Symbol('F_4') : sp.Symbol('F^(4)')/441,
#         sp.Symbol('F_0'): sp.Symbol('F^(0)'),
#         sp.Symbol('F_2'): sp.Symbol('F^(2)')/49}
# A = sp.Symbol('F_0') - 49 * sp.Symbol('F_4')
# A = A.subs(subs)
# B = sp.Symbol('F_2') - 5 * sp.Symbol('F_4')
# B = B.subs(subs)
# C = 35*sp.Symbol('F_4')
# C = C.subs(subs)
# comparison_brakets = {
# sp.Symbol(r'J(\zeta\zeta)'): (A + 4*B + 3*C, Qet({(sp.Symbol(r'{\xi}'),sp.Symbol(r'{\xi}'),sp.Symbol(r'{\xi}'),sp.Symbol(r'{\xi}')):1})),
# sp.Symbol(r'J(\xi\eta)'): (A - 2*B + C, Qet({(sp.Symbol(r'{\chi}'),sp.Symbol(r'{\phi}'),sp.Symbol(r'{\chi}'),sp.Symbol(r'{\phi}')):1})),
# sp.Symbol(r'J(\zeta{u})'): (A - 4*B + C, Qet({(sp.Symbol(r'{\xi}'),sp.Symbol(r'{\gamma}'),sp.Symbol(r'{\xi}'),sp.Symbol(r'{\gamma}')):1})),
# sp.Symbol(r'J(uv)'): (A - 4*B + C, Qet({(sp.Symbol(r'{\gamma}'),sp.Symbol(r'{\zeta}'),sp.Symbol(r'{\gamma}'),sp.Symbol(r'{\zeta}')):1})),
# sp.Symbol(r'J(\zeta{v})'): (A + 4*B + C, Qet({(sp.Symbol(r'{\xi}'),sp.Symbol(r'{\zeta}'),sp.Symbol(r'{\xi}'),sp.Symbol(r'{\zeta}')):1})),
# sp.Symbol(r'K(\xi\eta)'): (3*B + C, Qet({(sp.Symbol(r'{\chi}'),sp.Symbol(r'{\chi}'),sp.Symbol(r'{\phi}'),sp.Symbol(r'{\phi}')):1})),
# sp.Symbol(r'K(\zeta{u})'): (4*B + C, Qet({(sp.Symbol(r'{\xi}'),sp.Symbol(r'{\xi}'),sp.Symbol(r'{\gamma}'),sp.Symbol(r'{\gamma}')):1})),
# sp.Symbol(r'K(uv)'): (4*B + C, Qet({(sp.Symbol(r'{\gamma}'),sp.Symbol(r'{\gamma}'),sp.Symbol(r'{\zeta}'),sp.Symbol(r'{\zeta}')):1})),
# sp.Symbol(r'K(\zeta{v})'): (C, Qet({(sp.Symbol(r'{\xi}'),sp.Symbol(r'{\xi}'),sp.Symbol(r'{\zeta}'),sp.Symbol(r'{\zeta}')):1})),

# }
# # for k, v in comparison_brakets.items():
# #         display(k)
# #         gt =  sp.latex(v[0])
# #         display(v[0])
# #         abraket = v[1]
# #         matrix_element = simplifier(abraket)
# #         # out1 = r'%s' % (sp.latex(matrix_element.as_braket()))
# #         # display(Math(out1))
# #         matrix_element = braket_basis_change(matrix_element, basis_change)
# #         # display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
# #         # display(matrix_element.as_braket())
# #         matrix_element = matrix_element.apply(to_slater_params)
# #         out = r'%s' % (sp.latex(matrix_element.as_symbol_sum()))
# #         display(Math(out))
# #         print(gt==out)

# for k, v in comparison_brakets.items():
#         for k1, v1 in comparison_brakets.items():
#                 gt =  sp.latex(3*v[0]+2*v1[0])
#                 abraket = 3*v[1] + 2*v1[1]
#                 matrix_element = simplifier(abraket)
#                 # out1 = r'%s' % (sp.latex(matrix_element.as_braket()))
#                 # display(Math(out1))
#                 matrix_element = braket_basis_change(matrix_element, basis_change)
#                 # display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
#                 # display(matrix_element.as_braket())
#                 matrix_element = matrix_element.apply(to_slater_params)
#                 out = r'%s' % (sp.latex(matrix_element.as_symbol_sum()))
#                 if not(gt==out) or (np.random.randint(0,1) == 0):
#                         if (gt == out):
#                                 display(sp.Symbol('='))
#                         display(3*k+2*k1)
#                         # display(v[0]+2*v1[0])
#                         # display(Math(out))
#                         display(sp.Matrix([3*v[0]+2*v1[0], sp.Symbol('||'), sp.Symbol(out)]).T)

In [ ]:
# l = 2
# lp = 2
# m = 2
# mp = 1
# k = 2
# mk = m-mp
# yu = threeHarmonicIntegral(l,m,k,mk,lp,mp) * sp.sqrt(4*sp.pi) / sp.sqrt(2*k+1)
# yu

In [434]:
# Qet(to_slater_params((2, 2,
#                       2, -2,
#                       2, -2,
#                       2, 2),1)).as_symbol_sum()

In [397]:
# class Qot():
#     '''
#     A Qet is a dictionary of keys and values. Keys correspond to
#     tuples   of  quantum  numbers  or  symbols  and  the  values
#     correspond to the accompanying coefficients.

#     Scalars may be added by using an empty tuple as a key.

#     A  qet  may be multiplied by a scalar, in which case all the
#     coefficients are multiplied by it. It may also be multiplied
#     by   another   qet,   in  which  case  quantum  numbers  are
#     concatenated and coefficients multiplied accordingly.
#     '''
#     def __init__(self, bits=0):
#         if bits == 0:
#                 self.dict = {}
#         elif isinstance(bits, dict):
#                 self.dict = {k: v for k, v in bits.items() if v!=0}

#     def __add__(self, other):
#         new_dict = dict(self.dict)
#         if other == 0:
#                 return self
#         for key, coeff in other.dict.items():
#             if key in new_dict.keys():
#                 new_dict[key] += coeff
#             else:
#                 new_dict[key] = coeff
#         return Qet(new_dict)

#     def __sub__(self, other):
#         if other == 0:
#             return self
#         return Qet(dict(self.dict)) + (-1)*Qet(dict(other.dict))

#     def __neg__(self):
#         return (-1)*Qet(self.dict)

#     def vec_in_basis(self, basis):
#         '''
#         Given an ordered basis  return  a  list  with  the
#         coefficients of the qet in that basis.
#         '''
#         coeffs = [0]*len(basis)
#         for key, val in self.dict.items():
#             coeffs[basis.index(key)] = val
#         return coeffs

#     def subs(self, subs_dict):
#         '''
#         The substitutions in subs_dict are evaluated on the
#         coeffients of the qet.
#         Equal to valsubs but kept for backwards compatibility.
#         '''
#         new_dict = dict()
#         for key, val in self.dict.items():
#             new_dict[key] = sp.S(val).subs(subs_dict)
#         return Qet(new_dict)

#     def valsubs(self, subs_dict):
#         '''
#         The substitutions in subs_dict are evaluated on the
#         coeffients of the qet.
#         '''
#         new_dict = dict()
#         for key, val in self.dict.items():
#             new_dict[key] = sp.S(val).subs(subs_dict)
#         return Qet(new_dict)

#     def keysubs(self, subs_dict):
#         '''
#         The substitutions in subs_dict are evaluated on the
#         keys of the qet.
#         It assumes that they keys are already symby symbols.
#         If not, then they are converted to them.
#         '''
#         new_dict = dict()
#         for key, val in self.dict.items():
#             new_dict[sp.Symbol(key).subs(subs_dict)] = sp.S(val)
#         return Qet(new_dict)

#     def __mul__(self, multiplier):
#         '''
#         Give a representation of the qet  as  a  Ket  from
#         sympy.physics.quantum, fold_keys  =  True  removes
#         unnecessary parentheses and nice_negatives =  True
#         assumes all numeric  keys  and  presents  negative
#         values with a bar on top.
#         '''
#         if isinstance(multiplier, Qet):
#             new_dict = dict()
#             for k1, v1 in self.dict.items():
#                 for k2, v2 in multiplier.dict.items():
#                     k3 = k1 + k2
#                     v3 = v1 * v2
#                     if v3 !=0:
#                         new_dict[k3] = v3
#             return Qet(new_dict)
#         else:
#             new_dict = dict(self.dict)
#             for key, coeff in new_dict.items():
#                 new_dict[key] = multiplier*(coeff)
#             return Qet(new_dict)

#     def __rmul__(self, multiplier):
#         '''this is required to enable multiplication
#         from the left and from the right'''
#         new_dict = dict()
#         for key, coeff in self.dict.items():
#             new_dict[key] = multiplier*(coeff)
#         return Qet(new_dict)

#     def basis(self):
#         '''return a list with all the keys in the qet'''
#         return list(self.dict.keys())

#     def dual(self):
#         '''conjugate all the coefficients'''
#         new_dict = dict(self.dict)
#         for key, coeff in new_dict.items():
#             new_dict[key] = sp.conjugate(coeff)
#         return Qet(new_dict)

#     def as_operator(self, opfun):
#         OP = sp.S(0)
#         for key, val in self.dict.items():
#                 OP += sp.S(val) * opfun(*key)
#         return OP

#     def as_ket(self, fold_keys=False, nice_negatives=False):
#         '''
#         Give a representation of the qet  as  a  Ket  from
#         sympy.physics.quantum, fold_keys  =  True  removes
#         unnecessary parentheses and nice_negatives =  True
#         assumes all numeric  keys  and  presents  negative
#         values with a bar on top.
#         '''
#         sympyRep = sp.S(0)
#         for key, coeff in self.dict.items():
#             if key == ():
#                 sympyRep += coeff
#             else:
#                 if fold_keys:
#                     if nice_negatives:
#                         key = tuple(sp.latex(k) if k>=0 else (r'\bar{%s}'\
#                                                 % sp.latex(-k)) for k in key)
#                     sympyRep += coeff*Ket(*key)
#                 else:
#                     sympyRep += coeff*Ket(key)
#         return sympyRep

#     def as_bra(self):
#         '''
#         Give a representation of the qet  as  a  Bra  from
#         sympy.physics.quantum.
#         '''
#         sympyRep = sp.S(0)
#         for key, coeff in self.dict.items():
#             if key == ():
#                 sympyRep += coeff
#             else:
#                 sympyRep += coeff*Bra(*key)
#         return sympyRep

#     def as_braket(self):
#         '''
#         Give a representation of the qet as a Bra*Ket. The
#         keys in the dict for the ket are assumed to  split
#         first half for the bra, and other second half  for
#         the ket.
#         '''
#         sympyRep = sp.S(0)
#         for key, coeff in self.dict.items():
#             l = int(len(key)/2)
#             if key == ():
#                 sympyRep += coeff
#             else:
#                 sympyRep += coeff*(Bra(*key[:l])*Ket(*key[l:]))
#         return sympyRep

#     def as_symbol_sum(self):
#         '''
#         Take the keys multiply them by their corresponding
#         coefficients,  add  them  all  up,  and return the
#         resulting sympy expression.
#         '''
#         tot = sp.S(0)
#         for k, v in self.dict.items():
#             tot += v*k
#         return tot

#     def as_c_number_with_fun(self):
#         '''
#         This method can be used to apply a function  to  a
#         qet.  The  provided  function  f  must   take   as
#         arguments a single pair  of  qnum  and  coeff  and
#         return a dictionary or a (qnum, coeff) tuple.
#         '''
#         sympyRep = sp.S(0)
#         for key, op_and_coeff in self.dict.items():
#             ops_and_coeffs = list(zip(op_and_coeff[::2],op_and_coeff[1::2]))
#             for op, coeff in ops_and_coeffs:
#                 if key == ():
#                     sympyRep += coeff
#                 else:
#                     sympyRep += coeff*op(*key)
#         return sympyRep

#     def apply(self,f):
#         '''
#         This method can be used to apply a function  to  a
#         qet the provided function f must take as arguments
#         a single pair of  qnum  and  coeff  and  return  a
#         dictionary or a (qnum, coeff) tuple
#         '''
#         new_dict = dict()
#         for key, coeff in self.dict.items():
#             appfun = f(key,coeff)
#             if isinstance(appfun, dict):
#                 for key2, coeff2 in appfun.items():
#                     if coeff2 != 0:
#                         if key2 not in new_dict.keys():
#                             new_dict[key2] = coeff2
#                         else:
#                             new_dict[key2] += coeff2
#             else:
#                 new_key, new_coeff = appfun
#                 if new_coeff !=0:
#                     if new_key not in new_dict.keys():
#                         new_dict[new_key] = (new_coeff)
#                     else:
#                         new_dict[new_key] += (new_coeff)
#         return Qet(new_dict)

#     def norm(self):
#         '''compute the norm of the qet'''
#         norm2 = 0
#         for key, coeff in self.dict.items():
#             norm2 += abs(coeff)**2
#         return sp.sqrt(norm2)

#     def symmetrize(self):
#         '''
#         Use  if  the  keys  of the kets are tuples and one
#         wants  to  make equal keys that are the reverse of
#         one another, i.e.

#         {(1,0):a, (0,1):b} -> {(1,0):a+b}
#         '''
#         new_dict = dict()
#         for key, coeff in self.dict.items():
#             rkey = key[::-1]
#             if rkey in new_dict.keys():
#                 new_dict[rkey] += coeff
#             else:
#                 if key in new_dict.keys():
#                     new_dict[key] += coeff
#                 else:
#                     new_dict[key] = coeff
#         return Qet(new_dict)

#     def __str__(self):
#         return str(self.dict)

#     def __repr__(self):
#         return 'Qet(%s)' % str(self.dict)

# symmetry_bases = pickle.load(open(os.path.join(module_dir,'data',
#                                             'symmetry_bases.pkl'),'rb'))

# class ProductTable():
#     '''
#     This class used to hold the data of a direct product of two
#     irreducible representations turned  into a  direct  sum  of
#     irreducible representations.
#     '''
#     def __init__(self, odict, irrep_labels, grp_label):
#         self.odict = odict
#         self.irrep_labels = irrep_labels
#         self.grp_label = grp_label
#     def pretty_parse(self):
#         '''creates a nice latex representation of the product table'''
#         irep_symbols = self.irrep_labels
#         list_o_lists = [[self.odict[(ir0,ir1)] for ir0 in self.irrep_labels]\
#                                                 for ir1 in self.irrep_labels]
#         rows = [[sp.Symbol(self.grp_label)]+irep_symbols]
#         for idx, arow in enumerate(list_o_lists):
#             row = [irep_symbols[idx]]
#             row.extend(arow)
#             rows.append(row)
#         return fmt_table(rows).replace('+',r'{\oplus}')

In [405]:
def qfun(qnums, coeff):
    return {(1,1): coeff,(2,2):coeff}
qot = Qot({(1,1):1,(2,2):1}).apply(qfun)
qot.as_ket()

2*|(1, 1)> + 2*|(2, 2)>